In [50]:
maze = {}
with open("inputs/day20-input.txt") as f:
    for y, line in enumerate(f.read().splitlines()):
        for x, tile in enumerate(line):
            if not tile.isspace():
                maze[(x, y)] = tile
                
directions = {
    0: (0, -1),
    1: (1, 0),
    2: (0, 1),
    3: (-1, 0)
}

def move(position, direction):
    return (position[0] + direction[0], position[1] + direction[1])

positions = list(maze)
while positions:
    position = positions.pop()
    
    if maze.get(position, "").isalpha():
        
        if any(map(lambda x: maze.get(move(position, x), "") == ".", directions.values())):
            other_direction = next(filter(lambda x: maze.get(move(position, x), "").isalpha(), directions.values()))
            other = move(position, other_direction)
            
            label = "".join(sorted([maze[position], maze[other]]))
            maze[position] = label
            maze.pop(other)

## Part 1

In [54]:
%%time

from itertools import combinations

def inverse_direction(direction):
    if direction == (0, 0):
        return direction
    return directions[(next(x for x in directions if directions[x] == direction) + 2) % 4]

def backtrack(start, end, direction=(0, 0), steps=0, visited=set()):
    if start == end:
        return steps - 2 # Exclude start and end, as per problem statement we start one after and end one before
    
    visited.add(start)
    
    results = []
    back = inverse_direction(direction)
    for new_direction in directions.values():
        if new_direction == back:
            continue
            
        new_position = move(start, new_direction)
        if maze.get(new_position, "#") != "#":
            result = backtrack(new_position, end, new_direction, steps + 1, visited)
            
            if result:
                results.append(result)
            
    visited.remove(start)
    
    if results:
        return min(results)
    else:
        return False            


def backtrack_portals(start, end, steps=0, visited=set()):
    if start == end:
        return steps - 1 # No warp at last portal
    
    visited.add(start)
    
    results = []
    for n in graph[start]:
        if n in visited:
            continue
        
        new_steps = steps + graph[start][n] + 1 # +1 for the warp
        result = backtrack_portals(n, end, new_steps, visited)
        
        if result:
            results.append(result)
    
    visited.remove(start)
    
    if results:
        return min(results)
    else:
        return False


nodes = list(filter(lambda x: maze[x].isalpha(), maze))
graph = {maze[x]: {} for x in nodes}
for a, b in combinations(nodes, 2):
    if (length := backtrack(a, b)):
        node_a = maze[a]
        node_b = maze[b]
        graph[node_a][node_b] = length
        graph[node_b][node_a] = length
        
print(backtrack_portals("AA", "ZZ"))

656
CPU times: user 816 ms, sys: 2.27 ms, total: 819 ms
Wall time: 824 ms


## Part 2